In [ ]:
# install env
!pip3 install torch
!pip3 install torchvision

!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

In [ ]:
# test env
import torch
!/opt/bin/nvidia-smi

In [ ]:
# mount google drive to env

from google.colab import drive
import os
drive.mount('/content/drive')
data_dir = "/content/drive/My Drive/colab/data/"

In [ ]:
# tcn

import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size, padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

print("TCN component initialized")


In [ ]:
# model

from torch import nn
import torch.nn.functional as F


class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        output = self.tcn(x.transpose(1, 2)).transpose(1, 2)
        output = self.linear(output).float()
        return self.sig(output)


print("TCN model initialized")


In [ ]:
# data_generator

import torch
import numpy as np
from tqdm import tqdm

nan_mode = 1 # method to manage NaN value
data_dir = "../data/training/"


def read_psv(f):
  lines = f.readlines()
  m = np.empty((len(lines)-1, 41), dtype=np.float32)
  for i in range(1, len(lines)-1):
    line = lines[i][:-1].split('|')
    for j in range(len(line)):
      m[i][j] = np.float32(line[j])
  return m


def data_generator():

  file_name_pattern = 'p{:0=6}.psv'

  file_index = list(range(0, 2000)) # set A max file index is 20643
  # dataset_index = list(range(100000, 101000)) # set B max file index is 120000
  np.random.seed(12334)
  np.random.shuffle(file_index)
  dataset_index = [
    file_index[:round(len(file_index)*0.7)], # train_set_index
    file_index[round(len(file_index)*0.7):round(len(file_index)*0.85)], # valid_set_index
    file_index[round(len(file_index)*0.85):] # valid_set_index
  ]

  datas = [[], [], []]
  
  # read file
  for i in range(len(dataset_index)):
    for j in tqdm(range(len(dataset_index[i]))):
      file_name = file_name_pattern.format(j + 1)
      with open(data_dir + file_name) as datafile:
        datas[i].append(read_psv(datafile))
    
  for data in datas:
    for i in range(len(data)):
        data[i] = torch.Tensor(data[i].astype(np.float32))

  return datas[0], datas[1], datas[2]

print("data generator initialized")

In [ ]:
# sepsis.py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np


_cuda = True
_dropout = 0.25
_clip = 0.2
_epochs = 100
_ksize = 5
_levels = 4
_log_interval = 100
_lr = 1e-3
_optim = 'Adam'
_nhid = 150
_seed = 6783
input_size = 88

# Set the random seed manually for reproducibility.
torch.manual_seed(_seed)

X_train, X_valid, X_test = data_generator()

n_channels = [_nhid] * _levels
kernel_size = _ksize
dropout = _dropout

model = TCN(input_size, 1, n_channels, kernel_size, dropout=_dropout)


if _cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
lr = _lr
optimizer = getattr(optim, _optim)(model.parameters(), lr=lr)


def evaluate(X_data, name='Eval'):
    model.eval()
    eval_idx_list = np.arange(len(X_data), dtype="int32")
    total_loss = 0.0
    count = 0
    with torch.no_grad():
        for idx in eval_idx_list:
            data_line = X_data[idx]
            x, y = Variable(data_line[:-1]), Variable(data_line[1:])
            if _cuda:
                x, y = x.cuda(), y.cuda()
            output = model(x.unsqueeze(0)).squeeze(0)
            loss = -torch.trace(torch.matmul(y, torch.log(output).float().t()) +
                                torch.matmul((1-y), torch.log(1-output).float().t()))
            total_loss += loss.item()
            count += output.size(0)
        eval_loss = total_loss / count
        print(name + " loss: {:.5f}".format(eval_loss))
        return eval_loss


def train(ep):
    model.train()
    total_loss = 0
    count = 0
    train_idx_list = np.arange(len(X_train), dtype="int32")
    np.random.shuffle(train_idx_list)
    for idx in train_idx_list:
        data_line = X_train[idx]
        x, y = Variable(data_line[:-1]), Variable(data_line[1:])
        if _cuda:
            x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        output = model(x.unsqueeze(0)).squeeze(0)
        loss = -torch.trace(torch.matmul(y, torch.log(output).float().t()) +
                            torch.matmul((1 - y), torch.log(1 - output).float().t()))
        total_loss += loss.item()
        count += output.size(0)

        if _clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), _clip)
        loss.backward()
        optimizer.step()
        if idx > 0 and idx % _log_interval == 0:
            cur_loss = total_loss / count
            print("Epoch {:2d} | lr {:.5f} | loss {:.5f}".format(ep, lr, cur_loss))
            total_loss = 0.0
            count = 0

best_vloss = 1e8
vloss_list = []
model_name = "poly_music_{0}.pt".format(_data)
for ep in range(1, _epochs+1):
    train(ep)
    vloss = evaluate(X_valid, name='Validation')
    tloss = evaluate(X_test, name='Test')
    if vloss < best_vloss:
        with open(model_name, "wb") as f:
            torch.save(model, f)
            print("Saved model!\n")
        best_vloss = vloss
    if ep > 10 and vloss > max(vloss_list[-3:]):
        lr /= 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    vloss_list.append(vloss)

print('-' * 89)
model = torch.load(open(model_name, "rb"))
tloss = evaluate(X_test)